### Part 3: Take the Microwave Equivalent Snowpacks of Part 2 and calculate the priors for the MCMC model

Benoit Montpetit, CPS/CRD/ECCC, 2024  
Julien Meloche, CPS/CRD/ECCC, 2024  
Nicolas Leroux, RPN-E/MRD/ECCC, 2024  
Vincent Vionnet, RPN-E/MRD/ECCC, 2024  
Mike Brady, CPS/CRD/ECCC, 2024  

This notebook takes the SVS-2 outputs reduced to microwave equivalent snowpack (2-layer snowpacks) [(Meloche et al., Preprint)](https://egusphere.copernicus.org/preprints/2024/egusphere-2024-3169/) and calculates priors to use to initialize the MCMC model

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
from scipy.stats import truncnorm, norm

from matplotlib import pyplot as plt
import matplotlib

font = {'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)
plt.rcParams["axes.labelsize"] = 22
plt.rcParams["axes.labelweight"] = 'bold'
plt.rcParams['xtick.labelsize']=16
plt.rcParams['ytick.labelsize']=16

In [ ]:
svs_arctic_mes = xr.open_dataset('../Data/SVS-2_ArcticEnsembles_TVC02_MES.nc')

In [ ]:
svs_default_mes = xr.open_dataset('../Data/SVS-2_DefaultEnsembles_TVC02_MES.nc')

In [ ]:
umass = pd.read_pickle('../Data/UMass_TVC18-19_DB.pkl')

In [ ]:
umass_tvc02=umass.drop(columns='geometry')[(umass.radar_ts<=datetime.datetime(2019,2,1)) & (umass.radar_ts>=datetime.datetime(2019,1,1))]

In [ ]:
max_date = umass_tvc02.radar_ts.max()
min_date = umass_tvc02.radar_ts.min()

The file below can be processed from notebooks 1 to 6 found here in [TVCExp18-19](https://github.com/ECCCBen/TVCExp18-19) previously published in [Montpetit et al., 2024](https://doi.org/10.5194/tc-18-3857-2024)

In [ ]:
df_pits = pd.read_json('../Data/df_stat_pits.json')

In [ ]:
from constants import TVC02

In [ ]:
sites = pd.DataFrame({'site':TVC02})
sites.replace({'RS':'RP'}, regex=True, inplace=True)
sites=list(sites.site.values)
sites.remove('SC02')

In [ ]:
df_pits = df_pits[sites].T

The file below can be processed from notebooks 1 to 6 found here in [TVCExp18-19](https://github.com/ECCCBen/TVCExp18-19) previously published in [Montpetit et al., 2024](https://doi.org/10.5194/tc-18-3857-2024)

In [ ]:
df_aggpits=pd.read_json('../Data/df_aggpits.json')

In [ ]:
types = ['R','H']

h_snow = {}
h_snow['R'] = []
h_snow['H'] = []
h_snow['bulk'] = []
ssa = {}
ssa['R'] = []
ssa['H'] = []
ssa['bulk'] = []
rho = {}
rho['R'] = []
rho['H'] = []
rho['bulk'] = []
swe = {}
swe['R'] = []
swe['H'] = []
swe['bulk'] = []

snowpit_stats = pd.DataFrame(columns=['mean','median','std','quart1','quart3'], index = pd.MultiIndex.from_product([sites, ['density','ssa','swe','thickness'], types]))

for site in sites:

    if site.replace('RS', 'RP') in df_aggpits.keys():
    
        pits = df_aggpits.loc[:,site.replace('RS', 'RP')].dropna()

        temp_thick = {}
        temp_thick['H']=[]
        temp_thick['R']=[]
        temp_ssa = {}
        temp_ssa['H']=[]
        temp_ssa['R']=[]
        temp_rho = {}
        temp_rho['H']=[]
        temp_rho['R']=[]
        temp_swe = {}
        temp_swe['H']=[]
        temp_swe['R']=[]

        for pit in pits.keys():

            profile = pits[pit]

            if (np.array(profile['grain_type'])=='H').any()==True:

                for gtype in types:
                    h_snow[gtype].extend([np.array(profile['thickness'])[np.array(profile['grain_type'])==gtype].sum()])
                    ssa[gtype].extend(np.array(profile['ssa'])[np.array(profile['grain_type'])==gtype])
                    rho[gtype].extend(np.array(profile['density'])[np.array(profile['grain_type'])==gtype])
                    swe[gtype].extend([(np.array(profile['density'])[np.array(profile['grain_type'])==gtype]*
                                      np.array(profile['thickness'])[np.array(profile['grain_type'])==gtype]).sum()])

                    temp_thick[gtype].extend([np.array(profile['thickness'])[np.array(profile['grain_type'])==gtype].sum()])
                    temp_ssa[gtype].extend(np.array(profile['ssa'])[np.array(profile['grain_type'])==gtype])
                    temp_rho[gtype].extend(np.array(profile['density'])[np.array(profile['grain_type'])==gtype])
                    temp_swe[gtype].extend([(np.array(profile['density'])*
                                      np.array(profile['thickness'])).sum()])

                gtype='bulk'
                h_snow[gtype].extend([np.array(profile['thickness']).sum()])
                ssa[gtype].extend([np.average(np.array(profile['ssa']), weights=np.array(profile['thickness']))])
                rho[gtype].extend([np.average(np.array(profile['density']), weights=np.array(profile['thickness']))])
                swe[gtype].extend([(np.array(profile['density'])*
                                    np.array(profile['thickness'])).sum()])

            elif (np.array(profile['grain_type'])=='R').all()==True:

                gtype='R'

                h_snow[gtype].extend([np.array(profile['thickness']).sum()])
                ssa[gtype].extend(np.array(profile['ssa'])[np.array(profile['grain_type'])==gtype])
                rho[gtype].extend(np.array(profile['density'])[np.array(profile['grain_type'])==gtype])
                swe[gtype].extend([(np.array(profile['density'])[np.array(profile['grain_type'])==gtype]*
                                      np.array(profile['thickness'])[np.array(profile['grain_type'])==gtype]).sum()])

                temp_thick[gtype].extend([np.array(profile['thickness'])[np.array(profile['grain_type'])==gtype].sum()])
                temp_ssa[gtype].extend(np.array(profile['ssa'])[np.array(profile['grain_type'])==gtype])
                temp_rho[gtype].extend(np.array(profile['density'])[np.array(profile['grain_type'])==gtype])
                temp_swe[gtype].extend([(np.array(profile['density'])*
                                      np.array(profile['thickness'])).sum()])
                
                gtype='bulk'
                h_snow[gtype].extend([np.array(profile['thickness']).sum()])
                ssa[gtype].extend([np.average(np.array(profile['ssa']), weights=np.array(profile['thickness']))])
                rho[gtype].extend([np.average(np.array(profile['density']), weights=np.array(profile['thickness']))])
                swe[gtype].extend([(np.array(profile['density'])*
                                    np.array(profile['thickness'])).sum()])

        for gtype in ['H','R']:
            if len(temp_ssa[gtype])>0:
                snowpit_stats.loc[(site,'ssa',gtype),'mean']=np.array(temp_ssa[gtype]).mean()
                snowpit_stats.loc[(site,'ssa',gtype),'median']=np.median(np.array(temp_ssa[gtype]))
                snowpit_stats.loc[(site,'ssa',gtype),'std']=np.array(temp_ssa[gtype]).std()
                snowpit_stats.loc[(site,'ssa',gtype),'quart1']=np.quantile(np.array(temp_ssa[gtype]),0.25)
                snowpit_stats.loc[(site,'ssa',gtype),'quart3']=np.quantile(np.array(temp_ssa[gtype]),0.75)

                snowpit_stats.loc[(site,'density',gtype),'mean']=np.array(temp_rho[gtype]).mean()
                snowpit_stats.loc[(site,'density',gtype),'median']=np.median(np.array(temp_rho[gtype]))
                snowpit_stats.loc[(site,'density',gtype),'std']=np.array(temp_rho[gtype]).std()
                snowpit_stats.loc[(site,'density',gtype),'quart1']=np.quantile(np.array(temp_rho[gtype]),0.25)
                snowpit_stats.loc[(site,'density',gtype),'quart3']=np.quantile(np.array(temp_rho[gtype]),0.75)

                snowpit_stats.loc[(site,'thickness',gtype),'mean']=np.array(temp_thick[gtype]).mean()
                snowpit_stats.loc[(site,'thickness',gtype),'median']=np.median(np.array(temp_thick[gtype]))
                snowpit_stats.loc[(site,'thickness',gtype),'std']=np.array(temp_thick[gtype]).std()
                snowpit_stats.loc[(site,'thickness',gtype),'quart1']=np.quantile(np.array(temp_thick[gtype]),0.25)
                snowpit_stats.loc[(site,'thickness',gtype),'quart3']=np.quantile(np.array(temp_thick[gtype]),0.75)

                snowpit_stats.loc[(site,'swe',gtype),'mean']=np.array(temp_swe[gtype]).mean()
                snowpit_stats.loc[(site,'swe',gtype),'median']=np.median(np.array(temp_swe[gtype]))
                snowpit_stats.loc[(site,'swe',gtype),'std']=np.array(temp_swe[gtype]).std()
                snowpit_stats.loc[(site,'swe',gtype),'quart1']=np.quantile(np.array(temp_swe[gtype]),0.25)
                snowpit_stats.loc[(site,'swe',gtype),'quart3']=np.quantile(np.array(temp_swe[gtype]),0.75)

                
for gtype in ['H','R','bulk']:
    h_snow[gtype]=np.array(h_snow[gtype])
    ssa[gtype]=np.array(ssa[gtype])
    rho[gtype]=np.array(rho[gtype])
    swe[gtype]=np.array(swe[gtype])

# Saving the snowpit statistics to a NetCDF file to use for figures

In [ ]:
snowpit_stats_nc = xr.Dataset.from_dataframe(snowpit_stats)
snowpit_stats.to_netcdf('../Data/Snowpit_statistics.nc')

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(20,10))

x = np.linspace(0,1,200)
grain_types=['R','H','bulk']
lowers=np.array([0.05,0.05,0.05])
uppers=np.array([1,1,1])

for i in range(3):

    if i<2:
        loc_def=svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['thickness'].mean()
        scale_def=svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['thickness'].std()
    
        loc_arc=svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['thickness'].mean()
        scale_arc=svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['thickness'].std()
    
        a_def, b_def = (lowers[i] - loc_def) / scale_def, (uppers[i] - loc_def) / scale_def
        a_arc, b_arc = (lowers[i] - loc_arc) / scale_arc, (uppers[i] - loc_arc) / scale_arc
    
        ax[i].hist(svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['thickness'].values.flatten(), bins=np.arange(0,0.6,0.05),
                   color='k',density=True,histtype = "stepfilled",edgecolor='k',label='SVS-2\nDefault');
        ax[i].hist(svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['thickness'].values.flatten(), bins=np.arange(0,0.6,0.05),
                   color='grey',density=True,histtype = "stepfilled",edgecolor='k',alpha=0.7,label='SVS-2\nArctic');
        ax[i].hist(h_snow[grain_types[i]], bins=np.arange(0,0.6,0.05),color='cyan',density=True,alpha=0.5,histtype = "stepfilled",edgecolor='k',label='Snowpits');
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_def,scale=scale_def,a=a_def,b=b_def), 
                   color='magenta',label='Prior\nDefault', linewidth=2, linestyle='--');
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_arc,scale=scale_arc,a=a_arc,b=b_arc), 
                   color='magenta',label='Prior\nArctic', linewidth=2);
        ax[i].plot(x, norm.pdf(x,loc=h_snow[grain_types[i]].mean(),
                           scale=h_snow[grain_types[i]].std()), 
                   color='red',label='Snowpits', linewidth=2);
        ax[i].set_xlabel('H$_{snow}$ (m)');
        ax[i].set_xlim(0,0.6);
    else:
        loc_def=((svs_default_mes['thickness']).sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).mean()
        scale_def=((svs_default_mes['thickness']).sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).std()

        loc_arc=(svs_arctic_mes['thickness']).sum(dim='snow_layer').sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).mean()
        scale_arc=((svs_arctic_mes['thickness']).sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).std()
    
        a_def, b_def = (lowers[i] - loc_def) / scale_def, (uppers[i] - loc_def) / scale_def
        a_arc, b_arc = (lowers[i] - loc_arc) / scale_arc, (uppers[i] - loc_arc) / scale_arc
    
        ax[i].hist(((svs_default_mes['thickness']).sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).values.flatten(), bins=np.arange(0,0.6,0.05),
                   color='k',density=True,histtype = "stepfilled",edgecolor='k',label='SVS-2 Default');
        ax[i].hist(((svs_arctic_mes['thickness']).sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).values.flatten(), bins=np.arange(0,0.6,0.05),
                   color='grey',density=True,histtype = "stepfilled",edgecolor='k',alpha=0.7,label='SVS-2 Arctic');
        ax[i].hist(h_snow[grain_types[i]],bins=np.arange(0,0.6,0.05),color='cyan',density=True,alpha=0.5,histtype = "stepfilled",edgecolor='k',label='Snowpits');
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_def,scale=scale_def,a=a_def,b=b_def), 
                   color='magenta',label='Prior Default', linewidth=2, linestyle='--')
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_arc,scale=scale_arc,a=a_arc,b=b_arc), 
                   color='magenta',label='Prior Arctic', linewidth=2)
        ax[i].plot(x, norm.pdf(x,loc=h_snow[grain_types[i]].mean(),
                           scale=h_snow[grain_types[i]].std()), 
                   color='red',label='Snowpits', linewidth=2)
        ax[i].set_xlabel('H$_{snow}$ (m)');
        ax[i].set_xlim(0,0.6);
        
ax[1].legend();
ax[0].text(0.01, 9,'a)');
ax[1].text(0.01, 11,'b)');
ax[2].text(0.01, 16.5,'c)');
ax[0].set_ylabel('Probability Density Function');
fig.savefig('../Figures/fig03.png');

<center><img src="../Figures/fig03.png" Width="1000px"></center>

<center>Figure 3 of <a href=https://doi.org/10.5194/tc-18-3857-2024>(Montpetit et al., Preprint)</a>: Measured snow thickness (Hsnow) distributions during the January TVC campaign, compared to the thickness distributions provided by the 120 SVS-2 ensemble members <a href=https://doi.org/10.5194/tc-18-5685-2024>(Woolley et al., 2024)</a> for the two dominant snow grain type layers a) rounded grains, b) depth hoar, and c) the total thickness, including both snow layers. Magenta lines represent the truncated normal distributions (Table 1) used as priors, using the mean and standard deviations of the two different SVS-2 versions. The red line represents the normal distribution, using the mean and standard deviation of the measurements, and consists in the desired posterior distributions obtained by the MCMC approach.</center>

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(20,10))

x = np.linspace(100,450,200)
grain_types=['R','H','bulk']
lowers=np.array([150,100,100])
uppers=np.array([450,350,450])

for i in range(3):

    if i<2:
        loc_def=svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].mean()
        scale_def=svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].std()
    
        loc_arc=svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].mean()
        scale_arc=svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].std()
    
        a_def, b_def = (lowers[i] - loc_def) / scale_def, (uppers[i] - loc_def) / scale_def
        a_arc, b_arc = (lowers[i] - loc_arc) / scale_arc, (uppers[i] - loc_arc) / scale_arc
    
        ax[i].hist(svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].values.flatten(), bins=range(100,460,20),
                   color='k',density=True,histtype = "stepfilled",edgecolor='k',label='SVS-2\nDefault');
        ax[i].hist(svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].values.flatten(), bins=range(100,460,20),
                   color='grey',density=True,histtype = "stepfilled",edgecolor='k',alpha=0.7,label='SVS-2\nArctic');
        ax[i].hist(rho[grain_types[i]],bins=range(100,460,20),color='cyan',density=True,alpha=0.5,histtype = "stepfilled",edgecolor='k',label='Snowpits');
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_def,scale=scale_def,a=a_def,b=b_def), 
                   color='magenta',label='Prior\nDefault', linewidth=2, linestyle='--')
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_arc,scale=scale_arc,a=a_arc,b=b_arc), 
                   color='magenta',label='Prior\nArctic', linewidth=2)
        ax[i].plot(x, norm.pdf(x,loc=rho[grain_types[i]].mean(),
                           scale=rho[grain_types[i]].std()), 
                   color='red',label='Snowpits', linewidth=2)
        ax[i].set_xlabel('$\\rho_{snow} (kg \\cdot m^{-3})$');
        ax[i].set_xlim(100,450);
    else:
        loc_def=((svs_default_mes['SNODEN_ML']*svs_default_mes['thickness']).sum(dim='snow_layer')/
                 svs_default_mes['thickness'].sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).mean()
        scale_def=((svs_default_mes['SNODEN_ML']*svs_default_mes['thickness']).sum(dim='snow_layer')/
                   svs_default_mes['thickness'].sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).std()

        loc_arc=((svs_arctic_mes['SNODEN_ML']*svs_arctic_mes['thickness']).sum(dim='snow_layer')/
                 svs_arctic_mes['thickness'].sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).mean()
        scale_arc=((svs_arctic_mes['SNODEN_ML']*svs_arctic_mes['thickness']).sum(dim='snow_layer')/
                   svs_arctic_mes['thickness'].sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).std()
    
        a_def, b_def = (lowers[i] - loc_def) / scale_def, (uppers[i] - loc_def) / scale_def
        a_arc, b_arc = (lowers[i] - loc_arc) / scale_arc, (uppers[i] - loc_arc) / scale_arc
    
        ax[i].hist(((svs_default_mes['SNODEN_ML']*svs_default_mes['thickness']).sum(dim='snow_layer')/
                 svs_default_mes['thickness'].sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).values.flatten(), bins=range(100,460,20),
                   color='k',density=True,histtype = "stepfilled",edgecolor='k',label='SVS-2\nDefault');
        ax[i].hist(((svs_arctic_mes['SNODEN_ML']*svs_arctic_mes['thickness']).sum(dim='snow_layer')/
                 svs_arctic_mes['thickness'].sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).values.flatten(), bins=range(100,460,20),
                   color='grey',density=True,histtype = "stepfilled",edgecolor='k',alpha=0.7,label='SVS-2\nArctic');
        ax[i].hist(rho[grain_types[i]],bins=range(100,460,20),color='cyan',density=True,alpha=0.5,histtype = "stepfilled",edgecolor='k',label='Snowpits');
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_def,scale=scale_def,a=a_def,b=b_def), 
                   color='magenta',label='Prior\nDefault', linewidth=2, linestyle='--')
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_arc,scale=scale_arc,a=a_arc,b=b_arc), 
                   color='magenta',label='Prior\nArctic', linewidth=2)
        ax[i].plot(x, norm.pdf(x,loc=rho[grain_types[i]].mean(),
                           scale=rho[grain_types[i]].std()), 
                   color='red',label='Snowpits', linewidth=2)
        ax[i].set_xlabel('$\\rho_{snow} (kg \\cdot m^{-3})$');
        ax[i].set_xlim(100,450);

ax[2].hist(rho[grain_types[i]],bins=range(100,460,20),color='cyan',density=True,alpha=0.5,histtype = "stepfilled",edgecolor='k',label='Snowpits');
ax[1].legend();
ax[0].text(105, 0.0275,'a)');
ax[1].text(105, 0.035,'b)');
ax[2].text(105, 0.0248,'c)');
ax[0].set_ylabel('Probability Density Function');
fig.savefig('../Figures/fig04.png');

#### <center><img src="../Figures/fig04.png" Width="1000px"></center>

<center>Figure 4 of <a href=https://doi.org/10.5194/tc-18-3857-2024>(Montpetit et al., Preprint)</a>: Same as Figure 3 for snow density (ρsnow).</center>

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,10))

x = np.linspace(0,60,200)
grain_types=['R','H']
lowers=np.array([10,8])
uppers=np.array([50,25])

for i in range(2):

    loc_def=svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].mean()
    scale_def=svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].std()

    loc_arc=svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].mean()
    scale_arc=svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].std()

    a_def, b_def = (lowers[i] - loc_def) / scale_def, (uppers[i] - loc_def) / scale_def
    a_arc, b_arc = (lowers[i] - loc_arc) / scale_arc, (uppers[i] - loc_arc) / scale_arc

    ax[i].hist(svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].values.flatten(), bins=range(0,60,4),
               color='k',density=True,histtype = "stepfilled",edgecolor='k',label='SVS-2 Default');
    ax[i].hist(svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].values.flatten(), bins=range(0,60,4),
               color='grey',density=True,histtype = "stepfilled",edgecolor='k',alpha=0.7,label='SVS-2 Arcitc');
    ax[i].hist(ssa[grain_types[i]], bins=range(0,60,4),color='cyan',density=True,alpha=0.5,histtype = "stepfilled",edgecolor='k',label='Snowpits');
    ax[i].plot(x, truncnorm.pdf(x,loc=loc_def,scale=3*scale_def,a=a_def/3,b=b_def/3), 
               color='magenta',label='Prior Default', linewidth=2, linestyle='--');
    ax[i].plot(x, truncnorm.pdf(x,loc=loc_arc,scale=3*scale_arc,a=a_arc/3,b=b_arc/3), 
               color='magenta',label='Prior Arctic', linewidth=2);
    ax[i].plot(x, norm.pdf(x,loc=ssa[grain_types[i]].mean(),
                       scale=ssa[grain_types[i]].std()), 
               color='red',label='Snowpits', linewidth=2);
    ax[i].set_xlabel('$SSA (m^2kg^{-1})$');
    ax[i].set_xlim(0,55);
ax[0].legend();
ax[0].text(1, 0.17,'a)');
ax[1].text(1, 0.305,'b)');
ax[0].set_ylabel('Probability Density Function');
ax[1].set_xlim();
fig.savefig('../figures/fig05.png');

<center><img src="../Figures/fig05.png" Width="1000px"></center>

<center>Figure 5  of <a href=https://doi.org/10.5194/tc-18-3857-2024>(Montpetit et al., Preprint)</a>: Same as Figures 3 and 4 for SSA. Since SSA is grain type specific, a bulk snowpack value is not shown, as in Figures 3 and 4.</center>

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(20,10))

x = np.linspace(0,200,50)
grain_types=['R','H','bulk']
lowers=np.array([0,0,0])
uppers=np.array([200,200,200])

for i in range(3):

    if i<2:
        loc_def=svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)[['SNODEN_ML','thickness']].to_dataframe().apply(lambda x : x.iloc[1] * x.iloc[0], axis = 1).mean()
        scale_def=svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)[['SNODEN_ML','thickness']].to_dataframe().apply(lambda x : x.iloc[1] * x.iloc[0], axis = 1).std()
    
        loc_arc=svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)[['SNODEN_ML','thickness']].to_dataframe().apply(lambda x : x.iloc[1] * x.iloc[0], axis = 1).mean()
        scale_arc=svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)[['SNODEN_ML','thickness']].to_dataframe().apply(lambda x : x.iloc[1] * x.iloc[0], axis = 1).std()
    
        a_def, b_def = (lowers[i] - loc_def) / scale_def, (uppers[i] - loc_def) / scale_def
        a_arc, b_arc = (lowers[i] - loc_arc) / scale_arc, (uppers[i] - loc_arc) / scale_arc
    
        ax[i].hist(svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)[['SNODEN_ML','thickness']].to_dataframe().apply(lambda x : x.iloc[1] * x.iloc[0], axis = 1).values.flatten(), 
                   bins=np.arange(0,200,20), color='k',density=True,histtype = "stepfilled",edgecolor='k',label='SVS-2\nDefault');
        ax[i].hist(svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)[['SNODEN_ML','thickness']].to_dataframe().apply(lambda x : x.iloc[1] * x.iloc[0], axis = 1).values.flatten(), 
                   bins=np.arange(0,200,20), color='grey',density=True,histtype = "stepfilled",edgecolor='k',alpha=0.7,label='SVS-2\nArctic');
        ax[i].hist(swe[grain_types[i]], bins=np.arange(0,200,20),color='cyan',density=True,alpha=0.5,histtype = "stepfilled",edgecolor='k',label='Snowpits');
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_def,scale=scale_def,a=a_def,b=b_def), 
                   color='magenta',label='Prior\nDefault', linewidth=2, linestyle='--');
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_arc,scale=scale_arc,a=a_arc,b=b_arc), 
                   color='magenta',label='Prior\nArctic', linewidth=2);
        ax[i].plot(x, norm.pdf(x,loc=(swe[grain_types[i]]).mean(),
                           scale=(swe[grain_types[i]]).std()), 
                   color='red',label='Snowpits', linewidth=2);
        ax[i].set_xlabel('SWE (mm)');
        ax[i].set_xlim(0,200);
    else:
        loc_def=((svs_default_mes[['SNODEN_ML','thickness']]).sel(time=slice(
            min_date,max_date+datetime.timedelta(days=1))).to_dataframe().apply(
            lambda x : x.iloc[1] * x.iloc[0], axis = 1)).groupby(['ensemble','time']).sum().mean()
        scale_def=((svs_default_mes[['SNODEN_ML','thickness']]).sel(time=slice(
            min_date,max_date+datetime.timedelta(days=1))).to_dataframe().apply(
            lambda x : x.iloc[1] * x.iloc[0], axis = 1)).groupby(['ensemble','time']).sum().std()

        loc_arc=((svs_arctic_mes[['SNODEN_ML','thickness']]).sel(time=slice(
            min_date,max_date+datetime.timedelta(days=1))).to_dataframe().apply(
            lambda x : x.iloc[1] * x.iloc[0], axis = 1)).groupby(['ensemble','time']).sum().mean()
        scale_arc=((svs_arctic_mes[['SNODEN_ML','thickness']]).sel(time=slice(
            min_date,max_date+datetime.timedelta(days=1))).to_dataframe().apply(
            lambda x : x.iloc[1] * x.iloc[0], axis = 1)).groupby(['ensemble','time']).sum().std()
    
        a_def, b_def = (lowers[i] - loc_def) / scale_def, (uppers[i] - loc_def) / scale_def
        a_arc, b_arc = (lowers[i] - loc_arc) / scale_arc, (uppers[i] - loc_arc) / scale_arc
    
        ax[i].hist(((svs_default_mes[['SNODEN_ML','thickness']]).sel(time=slice(
            min_date,max_date+datetime.timedelta(days=1))).to_dataframe().apply(
            lambda x : x.iloc[1] * x.iloc[0], axis = 1)).groupby(['ensemble','time']).sum().values.flatten(), 
                   bins=np.arange(0,200,20), color='k',density=True,histtype = "stepfilled",edgecolor='k',label='SVS-2 Default');
        ax[i].hist(((svs_arctic_mes[['SNODEN_ML','thickness']]).sel(time=slice(
            min_date,max_date+datetime.timedelta(days=1))).to_dataframe().apply(
            lambda x : x.iloc[1] * x.iloc[0], axis = 1)).groupby(['ensemble','time']).sum().values.flatten(), 
                   bins=np.arange(0,200,20), color='grey',density=True,histtype = "stepfilled",edgecolor='k',alpha=0.7,label='SVS-2 Arctic');
        ax[i].hist(swe[grain_types[i]],bins=np.arange(0,200,20),color='cyan',density=True,alpha=0.5,histtype = "stepfilled",edgecolor='k',label='Snowpits');
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_def,scale=scale_def,a=a_def,b=b_def), 
                   color='magenta',label='Prior Default', linewidth=2, linestyle='--')
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_arc,scale=scale_arc,a=a_arc,b=b_arc), 
                   color='magenta',label='Prior Arctic', linewidth=2)
        ax[i].plot(x, norm.pdf(x,loc=swe[grain_types[i]].mean(),
                           scale=swe[grain_types[i]].std()), 
                   color='red',label='Snowpits', linewidth=2)
        ax[i].set_xlabel('SWE (mm)');
        ax[i].set_xlim(0,200);
        
ax[1].legend();
ax[0].text(5, 0.029,'a)');
ax[1].text(5, 0.038,'b)');
ax[2].text(5, 0.175,'c)');
ax[0].set_ylabel('Probability Density Function');
fig.savefig('../Figures/fig06.png');

<center><img src="../Figures/fig06.png" Width="1000px"></center>

<center>Figure 6  of <a href=https://doi.org/10.5194/tc-18-3857-2024>(Montpetit et al., Preprint)</a>: Same as Figures 3 and 4 for SWE.</center>

## Same as previous figures for the main meteorological weather station of TVC (see SM site in [Montpetit et al. (2024)](https://tc.copernicus.org/articles/18/3857/2024/))

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,10))

x = np.linspace(100,450,200)
grain_types=['R','H']
lowers=np.array([150,100])
uppers=np.array([450,350])

for i in range(2):

    loc_def=svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].mean()
    scale_def=svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].std()

    loc_arc=svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].mean()
    scale_arc=svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].std()

    a_def, b_def = (lowers[i] - loc_def) / scale_def, (uppers[i] - loc_def) / scale_def
    a_arc, b_arc = (lowers[i] - loc_arc) / scale_arc, (uppers[i] - loc_arc) / scale_arc

    ax[i].hist(svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].values.flatten(), bins=range(100,460,20),
               color='k',density=True,histtype = "stepfilled",edgecolor='k',label='SVS-2 Default');
    ax[i].hist(svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].values.flatten(), bins=range(100,460,20),
               color='grey',density=True,histtype = "stepfilled",edgecolor='k',alpha=0.7,label='SVS-2 Arctic');
    ax[i].hist(np.array(df_pits.loc['SM02'].density)[np.array(df_pits.loc['SM02'].grain_type)==grain_types[i]], bins=range(100,460,20),
               density=True,color='cyan',alpha=0.5,histtype = "stepfilled",edgecolor='k',label='Snowpit SM');
    ax[i].plot(x, truncnorm.pdf(x,loc=loc_def,scale=scale_def,a=a_def,b=b_def), 
               color='magenta',label='Prior Default', linewidth=2, linestyle='--');
    ax[i].plot(x, truncnorm.pdf(x,loc=loc_arc,scale=scale_arc,a=a_arc,b=b_arc), 
               color='magenta',label='Prior Arctic', linewidth=2);
    ax[i].plot(x, norm.pdf(x,loc=np.array(df_pits.loc['SM02'].density)[np.array(df_pits.loc['SM02'].grain_type)==grain_types[i]].mean(),
                       scale=np.array(df_pits.loc['SM02'].density)[np.array(df_pits.loc['SM02'].grain_type)==grain_types[i]].std()), 
               color='red',label='Posterior', linewidth=2);
    ax[i].set_xlabel('$\\rho_{snow} (kg \\cdot m^{-3})$');

ax[1].legend();
ax[0].set_title('Wind Slab');
ax[1].set_title('Depth Hoar');
ax[0].set_ylabel('Probability Density Function');
fig.savefig('../../figures/PitVsSVS_SM_Density.png');

<center><img src="../Figures/PitVsSVS_SM_Density.png" Width="1000px"></center>

<center>Figure: Snow density prior distributions of the SM site <a href="https://tc.copernicus.org/articles/18/3857/2024/"> (see Table 2 of Montpetit et al., 2024)</a> of the default and Arctic versions of SVS-2 for all 120 ensemble members <a href="https://tc.copernicus.org/articles/18/5685/2024/">(Woolley et al., 2024)</a> for a) the rounded grain layer (R), and b) the depth hoar layer (DH).</center>

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,10))

x = np.linspace(0,60,200)
grain_types=['R','H']
lowers=np.array([10,8])
uppers=np.array([50,25])

for i in range(2):

    loc_def=svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].mean()
    scale_def=svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].std()

    loc_arc=svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].mean()
    scale_arc=svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].std()

    a_def, b_def = (lowers[i] - loc_def) / scale_def, (uppers[i] - loc_def) / scale_def
    a_arc, b_arc = (lowers[i] - loc_arc) / scale_arc, (uppers[i] - loc_arc) / scale_arc

    ax[i].hist(svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].values.flatten(), bins=range(0,60,4),
               color='k',density=True,histtype = "stepfilled",edgecolor='k',label='SVS-2 Default');
    ax[i].hist(svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].values.flatten(), bins=range(0,60,4),
               color='grey',density=True,histtype = "stepfilled",edgecolor='k',alpha=0.7,label='SVS-2 Arctic');
    ax[i].hist(np.array(df_pits.loc['SM02'].ssa)[np.array(df_pits.loc['SM02'].grain_type)==grain_types[i]], bins=range(0,60,4),
               density=True,color='cyan',alpha=0.5,histtype = "stepfilled",edgecolor='k',label='Snowpit SM');
    ax[i].plot(x, truncnorm.pdf(x,loc=loc_def,scale=scale_def,a=a_def,b=b_def), 
               color='magenta',label='Prior Default', linewidth=2, linestyle='--');
    ax[i].plot(x, truncnorm.pdf(x,loc=loc_arc,scale=scale_arc,a=a_arc,b=b_arc), 
               color='magenta',label='Prior Arctic', linewidth=2);
    ax[i].plot(x, norm.pdf(x,loc=np.array(df_pits.loc['SM02'].ssa)[np.array(df_pits.loc['SM02'].grain_type)==grain_types[i]].mean(),
                       scale=np.array(df_pits.loc['SM02'].ssa)[np.array(df_pits.loc['SM02'].grain_type)==grain_types[i]].std()), 
               color='red',label='Posterior', linewidth=2)
    ax[i].set_xlabel('$SSA (m^2kg^{-1})$');

ax[1].legend();
ax[0].set_title('Wind Slab');
ax[1].set_title('Depth Hoar');
ax[0].set_ylabel('Probability Density Function');
fig.savefig('../../figures/PitVsSVS_SM_SSA.png');

<center><img src="../Figures/PitVsSVS_SM_SSA.png" Width="1000px"></center>

<center>Figure: SSA prior distributions of the SM site <a href="https://tc.copernicus.org/articles/18/3857/2024/"> (see Table 2 of Montpetit et al., 2024)</a> of the default and Arctic versions of SVS-2 for all 120 ensemble members <a href="https://tc.copernicus.org/articles/18/5685/2024/">(Woolley et al., 2024)</a> for a) the rounded grain layer (R), and b) the depth hoar layer (DH).</center>

## Creating the stats to generate the priors from all 120 ensemble members

In [ ]:
variables = []
graintype = []
means=[]
stds=[]
site_id=[]

grain_types=['R','H']

for varia in ['thickness','SNODEN_ML','ssa']:
    for i in range(2):
        if varia=='SNODEN_ML':
            variables.append('density')
        else:
            variables.append(varia)
        graintype.append(grain_types[i])
        means.append([svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)[varia].mean().values,
                      svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)[varia].mean().values])
        stds.append([svs_default_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)[varia].std().values,
                    svs_arctic_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)[varia].std().values])

indexes = pd.MultiIndex.from_arrays([variables, graintype], names=('property','grain_type'))

mins=np.array([0.05,0.05,150,100,10,8])
maxs=np.array([1,1,450,350,50,25])

df_priors_default = pd.DataFrame({'mean':np.array(means)[:,0],'std':np.array(stds)[:,0],'min':mins,'max':maxs}, index=indexes)
df_priors_arctic = pd.DataFrame({'mean':np.array(means)[:,1],'std':np.array(stds)[:,1],'min':mins,'max':maxs}, index=indexes)

In [ ]:
xr.Dataset.from_dataframe(df_priors_default).to_netcdf('../Data/SVS-2_DefaultPriors.nc')
xr.Dataset.from_dataframe(df_priors_arctic).to_netcdf('../Data/SVS-2_ArcticPriors.nc')

# Calculate priors using the Top 30 ensemble members

In [ ]:
svs_arctictop_mes = xr.open_dataset('../Data/SVS-2_ArcticTop30Ensembles_TVC02_MES.nc')
svs_defaulttop_mes = xr.open_dataset('../Data/SVS-2_DefaultTop30Ensembles_TVC02_MES.nc')

In [ ]:
variables = []
graintype = []
means=[]
stds=[]
site_id=[]

grain_types=['R','H']

for varia in ['thickness','SNODEN_ML','ssa']:
    for i in range(2):
        if varia=='SNODEN_ML':
            variables.append('density')
        else:
            variables.append(varia)
        graintype.append(grain_types[i])
        means.append([svs_defaulttop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)[varia].mean().values,
                      svs_arctictop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)[varia].mean().values])
        stds.append([svs_defaulttop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)[varia].std().values,
                    svs_arctictop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)[varia].std().values])

indexes = pd.MultiIndex.from_arrays([variables, graintype], names=('property','grain_type'))

mins=np.array([0.05,0.05,150,100,10,8])
maxs=np.array([1,1,450,350,50,25])

df_priors_default = pd.DataFrame({'mean':np.array(means)[:,0],'std':np.array(stds)[:,0],'min':mins,'max':maxs}, index=indexes)
df_priors_arctic = pd.DataFrame({'mean':np.array(means)[:,1],'std':np.array(stds)[:,1],'min':mins,'max':maxs}, index=indexes)

In [ ]:
xr.Dataset.from_dataframe(df_priors_default).to_netcdf('../Data/SVS-2_DefaultPriorsTop30.nc')
xr.Dataset.from_dataframe(df_priors_arctic).to_netcdf('../Data/SVS-2_ArcticPriorsTop30.nc')

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(20,10))

x = np.linspace(100,450,200)
grain_types=['R','H','bulk']
lowers=np.array([150,100,100])
uppers=np.array([450,350,450])

for i in range(3):

    if i<2:
        loc_def=svs_defaulttop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].mean()
        scale_def=svs_defaulttop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].std()
    
        loc_arc=svs_arctictop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].mean()
        scale_arc=svs_arctictop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].std()
    
        a_def, b_def = (lowers[i] - loc_def) / scale_def, (uppers[i] - loc_def) / scale_def
        a_arc, b_arc = (lowers[i] - loc_arc) / scale_arc, (uppers[i] - loc_arc) / scale_arc
    
        ax[i].hist(svs_defaulttop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].values.flatten(), bins=range(100,460,20),
                   color='k',density=True,histtype = "stepfilled",edgecolor='k',label='SVS-2\nDefault');
        ax[i].hist(svs_arctictop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['SNODEN_ML'].values.flatten(), bins=range(100,460,20),
                   color='grey',density=True,histtype = "stepfilled",edgecolor='k',alpha=0.7,label='SVS-2\nArctic');
        ax[i].hist(rho[grain_types[i]],bins=range(100,460,20),color='cyan',density=True,alpha=0.5,histtype = "stepfilled",edgecolor='k',label='Snowpits');
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_def,scale=scale_def,a=a_def,b=b_def), 
                   color='magenta',label='Prior\nDefault', linewidth=2, linestyle='--')
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_arc,scale=scale_arc,a=a_arc,b=b_arc), 
                   color='magenta',label='Prior\nArctic', linewidth=2)
        ax[i].plot(x, norm.pdf(x,loc=rho[grain_types[i]].mean(),
                           scale=rho[grain_types[i]].std()), 
                   color='red',label='Posterior', linewidth=2)
        ax[i].set_xlabel('$\\rho_{snow} (kg \\cdot m^{-3})$');
        ax[i].set_xlim(100,450);
    else:
        loc_def=((svs_defaulttop_mes['SNODEN_ML']*svs_defaulttop_mes['thickness']).sum(dim='snow_layer')/
                 svs_defaulttop_mes['thickness'].sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).mean()
        scale_def=((svs_defaulttop_mes['SNODEN_ML']*svs_defaulttop_mes['thickness']).sum(dim='snow_layer')/
                   svs_defaulttop_mes['thickness'].sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).std()

        loc_arc=((svs_arctictop_mes['SNODEN_ML']*svs_arctictop_mes['thickness']).sum(dim='snow_layer')/
                 svs_arctictop_mes['thickness'].sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).mean()
        scale_arc=((svs_arctictop_mes['SNODEN_ML']*svs_arctictop_mes['thickness']).sum(dim='snow_layer')/
                   svs_arctictop_mes['thickness'].sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).std()
    
        a_def, b_def = (lowers[i] - loc_def) / scale_def, (uppers[i] - loc_def) / scale_def
        a_arc, b_arc = (lowers[i] - loc_arc) / scale_arc, (uppers[i] - loc_arc) / scale_arc
    
        ax[i].hist(((svs_defaulttop_mes['SNODEN_ML']*svs_defaulttop_mes['thickness']).sum(dim='snow_layer')/
                 svs_defaulttop_mes['thickness'].sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).values.flatten(), bins=range(100,460,20),
                   color='k',density=True,histtype = "stepfilled",edgecolor='k',label='SVS-2\nDefault');
        ax[i].hist(((svs_arctictop_mes['SNODEN_ML']*svs_arctictop_mes['thickness']).sum(dim='snow_layer')/
                 svs_arctictop_mes['thickness'].sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).values.flatten(), bins=range(100,460,20),
                   color='grey',density=True,histtype = "stepfilled",edgecolor='k',alpha=0.7,label='SVS-2\nArctic');
        ax[i].hist(rho[grain_types[i]],bins=range(100,460,20),color='cyan',density=True,alpha=0.5,histtype = "stepfilled",edgecolor='k',label='Snowpits');
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_def,scale=scale_def,a=a_def,b=b_def), 
                   color='magenta',label='Prior\nDefault', linewidth=2, linestyle='--')
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_arc,scale=scale_arc,a=a_arc,b=b_arc), 
                   color='magenta',label='Prior\nArctic', linewidth=2)
        ax[i].plot(x, norm.pdf(x,loc=rho[grain_types[i]].mean(),
                           scale=rho[grain_types[i]].std()), 
                   color='red',label='Posterior', linewidth=2)
        ax[i].set_xlabel('$\\rho_{snow} (kg \\cdot m^{-3})$');
        ax[i].set_xlim(100,450);

ax[2].hist(rho[grain_types[i]],bins=range(100,460,20),color='cyan',density=True,alpha=0.5,histtype = "stepfilled",edgecolor='k',label='Snowpits');
ax[1].legend();
ax[0].text(105, 0.075,'a)');
ax[1].text(105, 0.12,'b)');
ax[2].text(105, 0.11,'c)');
ax[0].set_ylabel('Probability Density Function');
fig.savefig('../Figures/PitVsSVS_Top30_density.png');

<center><img src="../Figures/PitVsSVS_Top30_density.png" Width="1000px"></center>

<center>Figure: Snow density prior distributions of the default and Arctic versions of SVS-2 for the top 30 ensemble members <a href="https://tc.copernicus.org/articles/18/5685/2024/">(Woolley et al., 2024)</a> for a) the rounded grain layer (R), b) the depth hoar layer (DH), and c) the bulk density.</center>

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(20,10))

x = np.linspace(0,60,200)
grain_types=['R','H']
lowers=np.array([10,8])
uppers=np.array([50,25])

for i in range(2):

    loc_def=svs_defaulttop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].mean()
    scale_def=svs_defaulttop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].std()

    loc_arc=svs_arctictop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].mean()
    scale_arc=svs_arctictop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].std()

    a_def, b_def = (lowers[i] - loc_def) / scale_def, (uppers[i] - loc_def) / scale_def
    a_arc, b_arc = (lowers[i] - loc_arc) / scale_arc, (uppers[i] - loc_arc) / scale_arc

    ax[i].hist(svs_defaulttop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].values.flatten(), bins=range(0,60,4),
               color='k',density=True,histtype = "stepfilled",edgecolor='k',label='SVS-2 Default');
    ax[i].hist(svs_arctictop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['ssa'].values.flatten(), bins=range(0,60,4),
               color='grey',density=True,histtype = "stepfilled",edgecolor='k',alpha=0.7,label='SVS-2 Arcitc');
    ax[i].hist(ssa[grain_types[i]], bins=range(0,60,4),color='cyan',density=True,alpha=0.5,histtype = "stepfilled",edgecolor='k',label='Snowpits');
    ax[i].plot(x, truncnorm.pdf(x,loc=loc_def,scale=3*scale_def,a=a_def/3,b=b_def/3), 
               color='magenta',label='Prior Default', linewidth=2, linestyle='--');
    ax[i].plot(x, truncnorm.pdf(x,loc=loc_arc,scale=3*scale_arc,a=a_arc/3,b=b_arc/3), 
               color='magenta',label='Prior Arctic', linewidth=2);
    ax[i].plot(x, norm.pdf(x,loc=ssa[grain_types[i]].mean(),
                       scale=ssa[grain_types[i]].std()), 
               color='red',label='Posterior', linewidth=2);
    ax[i].set_xlabel('$SSA (m^2kg^{-1})$');
    ax[i].set_xlim(0,55);
ax[0].legend();
ax[0].text(1, 0.17,'a)');
ax[1].text(1, 0.305,'b)');
ax[0].set_ylabel('Probability Density Function');
ax[1].set_xlim();
fig.savefig('../Figures/PitVsSVS_top30_ssa.png');

<center><img src="../Figures/PitVsSVS_top30_ssa.png" Width="1000px"></center>

<center>Figure: SSA prior distributions of the default and Arctic versions of SVS-2 for the top 30 ensemble members <a href="https://tc.copernicus.org/articles/18/5685/2024/">(Woolley et al., 2024)</a> for a) the rounded grain layer (R), and b) the depth hoar layer (DH).</center>

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(20,10))

x = np.linspace(0,1,200)
grain_types=['R','H','bulk']
lowers=np.array([0.05,0.05,0.05])
uppers=np.array([1,1,1])

for i in range(3):

    if i<2:
        loc_def=svs_defaulttop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['thickness'].mean()
        scale_def=svs_defaulttop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['thickness'].std()
    
        loc_arc=svs_arctictop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['thickness'].mean()
        scale_arc=svs_arctictop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['thickness'].std()
    
        a_def, b_def = (lowers[i] - loc_def) / scale_def, (uppers[i] - loc_def) / scale_def
        a_arc, b_arc = (lowers[i] - loc_arc) / scale_arc, (uppers[i] - loc_arc) / scale_arc
    
        ax[i].hist(svs_defaulttop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['thickness'].values.flatten(), bins=np.arange(0,0.6,0.05),
                   color='k',density=True,histtype = "stepfilled",edgecolor='k',label='SVS-2\nDefault');
        ax[i].hist(svs_arctictop_mes.sel(time=slice(min_date,max_date+datetime.timedelta(days=1)), snow_layer=i)['thickness'].values.flatten(), bins=np.arange(0,0.6,0.05),
                   color='grey',density=True,histtype = "stepfilled",edgecolor='k',alpha=0.7,label='SVS-2\nArctic');
        ax[i].hist(h_snow[grain_types[i]], bins=np.arange(0,0.6,0.05),color='cyan',density=True,alpha=0.5,histtype = "stepfilled",edgecolor='k',label='Snowpits');
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_def,scale=scale_def,a=a_def,b=b_def), 
                   color='magenta',label='Prior\nDefault', linewidth=2, linestyle='--');
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_arc,scale=scale_arc,a=a_arc,b=b_arc), 
                   color='magenta',label='Prior\nArctic', linewidth=2);
        ax[i].plot(x, norm.pdf(x,loc=h_snow[grain_types[i]].mean(),
                           scale=h_snow[grain_types[i]].std()), 
                   color='red',label='Posterior', linewidth=2);
        ax[i].set_xlabel('H$_{snow}$ (m)');
        ax[i].set_xlim(0,0.6);
    else:
        loc_def=((svs_defaulttop_mes['thickness']).sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).mean()
        scale_def=((svs_defaulttop_mes['thickness']).sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).std()

        loc_arc=(svs_arctictop_mes['thickness']).sum(dim='snow_layer').sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).mean()
        scale_arc=((svs_arctictop_mes['thickness']).sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).std()
    
        a_def, b_def = (lowers[i] - loc_def) / scale_def, (uppers[i] - loc_def) / scale_def
        a_arc, b_arc = (lowers[i] - loc_arc) / scale_arc, (uppers[i] - loc_arc) / scale_arc
    
        ax[i].hist(((svs_defaulttop_mes['thickness']).sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).values.flatten(), bins=np.arange(0,0.6,0.05),
                   color='k',density=True,histtype = "stepfilled",edgecolor='k',label='SVS-2 Default');
        ax[i].hist(((svs_arctictop_mes['thickness']).sum(dim='snow_layer')).sel(time=slice(min_date,max_date+datetime.timedelta(days=1))).values.flatten(), bins=np.arange(0,0.6,0.05),
                   color='grey',density=True,histtype = "stepfilled",edgecolor='k',alpha=0.7,label='SVS-2 Arctic');
        ax[i].hist(h_snow[grain_types[i]],bins=np.arange(0,0.6,0.05),color='cyan',density=True,alpha=0.5,histtype = "stepfilled",edgecolor='k',label='Snowpits');
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_def,scale=scale_def,a=a_def,b=b_def), 
                   color='magenta',label='Prior Default', linewidth=2, linestyle='--')
        ax[i].plot(x, truncnorm.pdf(x,loc=loc_arc,scale=scale_arc,a=a_arc,b=b_arc), 
                   color='magenta',label='Prior Arctic', linewidth=2)
        ax[i].plot(x, norm.pdf(x,loc=h_snow[grain_types[i]].mean(),
                           scale=h_snow[grain_types[i]].std()), 
                   color='red',label='Posterior', linewidth=2)
        ax[i].set_xlabel('H$_{snow}$ (m)');
        ax[i].set_xlim(0,0.6);
        
ax[1].legend();
ax[0].text(0.01, 9,'a)');
ax[1].text(0.01, 11,'b)');
ax[2].text(0.01, 16.5,'c)');
ax[0].set_ylabel('Probability Density Function');
fig.savefig('../Figures/PitVsSVS_top30_thick.png');

<center><img src="../Figures/PitVsSVS_top30_thick.png" Width="1000px"></center>

<center>Figure: Snow height prior distributions of the default and Arctic versions of SVS-2 for the top 30 ensemble members <a href="https://tc.copernicus.org/articles/18/5685/2024/">(Woolley et al., 2024)</a> for a) the rounded grain layer (R), b) the depth hoar layer (DH), and c) the total depth .</center>